### BDM-3035 BIG DATA CAPSTONE PROJECT
* WILDFIRE PREDICTION
* Load the CWFIS GIS data

In [7]:
!pip install pyshp

  Obtaining dependency information for pyshp from https://files.pythonhosted.org/packages/98/2f/68116db5b36b895c0450e3072b8cb6c2fac0359279b182ea97014d3c8ac0/pyshp-2.3.1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ------------------------------------ --- 51.2/56.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 56.0/56.0 kB 737.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
   -------------------------- ------------- 30.7/46.5 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 46.5/46.5 kB 772.3 kB/s eta 0:00:00


### 1. Steps to gather and describe FWI/FBP geographic data structure for a specific year

The data structure can be understood by examining the shapefile according to data from the Canadian Forest Fire Weather Index (FWI) System and the Fire Behavior Prediction (FBP) System. The FWI System provides a comprehensive background on how weather conditions influence forest fires, with detailed information available at FWI System. Similarly, the FBP System offers insights into fire behavior predictions, with more details accessible at FBP System. Together, these systems provide crucial data for analyzing and predicting forest fire behavior in Canada. 

* Stages:
1. Download the file 2019_hotspots.zip for the year 2019 available at https://cwfis.cfs.nrcan.gc.ca/downloads/hotspots/archive/
2. Extract the shpae file 2019_hotspots.shp to local path './data/fwi_fbp/2019_hotspots/' from 2019_hotspots.zip file     
2. Gather FWI/FBP historical data for the year 2019
3. Convert shape file and record to GeoJSON dictionary
4. Create GeoJSON feature collection
5. Print the first feature in GeoJSON format 

* References:

1. Canadian Forest Service. Background information for the Fire Weather Index (FWI) System. Retrieved from https://cwfis.cfs.nrcan.gc.ca/background/summary/fwi

2. Canadian Forest Service. Background information for the Fire Behavior Prediction (FBP) System. Retrieved from https://cwfis.cfs.nrcan.gc.ca/background/summary/fbp

In [17]:
import shapefile
import json

# Path to your shapefile
shp_path = './data/fwi_fbp/2019_hotspots/2019_hotspots.shp'

# Read the shapefile
reader = shapefile.Reader(shp_path)

# Function to convert shapefile and record to GeoJSON format
def shape_record_to_geojson(shape_record):
    shape = shape_record.shape
    record = shape_record.record.as_dict()  # Convert the record to a dictionary
    return {
        "type": "Feature",
        "geometry": shape.__geo_interface__,
        "properties": record
    }

# Convert all shapes and records to GeoJSON features
features = [shape_record_to_geojson(sr) for sr in reader.shapeRecords()]

# Create GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Print the first feature in GeoJSON format
print(json.dumps(geojson['features'][0], indent=2))

{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      434060.17777945794,
      -645442.6049185912
    ]
  },
  "properties": {
    "lat": 43.109,
    "lon": -89.802,
    "rep_date": "2019/03/26 17:10:00.000",
    "source": "NASA",
    "sensor": "MODIS",
    "satellite": "Terra",
    "agency": "UWI",
    "temp": 8.454,
    "rh": 49,
    "ws": 14.785,
    "wd": 30,
    "pcp": 0.769,
    "ffmc": 74.33,
    "dmc": 0.0,
    "dc": 67.248,
    "isi": 1.558,
    "bui": 0.0,
    "fwi": 0.312,
    "fuel": "O1a",
    "ros": 2.64016,
    "sfc": 0.0,
    "tfc": 0.0,
    "bfc": 0.0690427,
    "hfi": 0,
    "cfb": 0,
    "estarea": 50.098,
    "pcuring": 100,
    "greenup": 0,
    "elev": 291,
    "sfl": 1.84599,
    "cfl": 0.0,
    "tfc0": null,
    "ecozone": "",
    "sfc0": null,
    "cbh": null
  }
}


### 2. Gather images of maps for a specific year 
The images can be gathered by using the file naming convention, as follows: 
1. Maptype (feature name) + yyyymmdd (eg: fwi20140521, ft20140521). For instance, the links could be  https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/fwi20140521.png and https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/ft20140521.png

2. Description of Maptype feature names:
* fwi: it does represent Canadian Fire Weather Index (FWI) at https://cwfis.cfs.nrcan.gc.ca/maps/fw?type=fwi
* ft: it does represent Fire Type of Canadian Forest Fire Behavior Prediction (FBP) System at https://cwfis.cfs.nrcan.gc.ca/maps/fb?type=ft

In [4]:
import requests
import os

# List of feature names
feature_names = ["fwi", "ft"]

# Function to download images for a specific year
def download_images_for_year(base_url, year, base_dir):
    for month in range(1, 13):
        for day in range(1, 32):
            date = f"{year}{month:02d}{day:02d}"  # Pad month and day with zero if necessary
            for feature_name in feature_names:
                image_url = f"{base_url}/{year}/{feature_name}{date}.png"
                response = requests.get(image_url)
                if response.status_code == 200:
                    # Create directory if it doesn't exist
                    os.makedirs(os.path.join(base_dir, feature_name), exist_ok=True)
                    # Save the image
                    with open(os.path.join(base_dir, feature_name, f"{feature_name}{date}.png"), 'wb') as f:
                        f.write(response.content)
#                     print(f"Downloaded image: {feature_name}{date}.png")
                else:
                    print(f"Failed to download image: {image_url}")

# Base URL for the images
base_url = "https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp"

# Year
year = "2014"

# Base directory for saving images
base_dir = './data/fwi_fbp/images'

# Call the function to download images for the year 2014
download_images_for_year(base_url, year, base_dir)

Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/fwi20140101.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/ft20140101.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/fwi20140102.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/ft20140102.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/fwi20140103.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/ft20140103.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/fwi20140104.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/ft20140104.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/fwi20140105.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca/data/maps/fwi_fbp/2014/ft20140105.png
Failed to download image: https://cwfis.cfs.nrcan.gc.ca

### 3. Using WMS service for requesting Maps of each feature, such as Fire Danger, Fire Weather Index, and Fire Type

* Requesting service maps to download Fire Danger, Fire Weather Index and Fire Type files 
* Please consider the documentation available at https://cwfis.cfs.nrcan.gc.ca/downloads/CWFIS_DataServices_HowtoAccessDailyMaps&DataLayers.pdf for WMS service

In [10]:
import requests
import os

# Function to download and save image from URL
def download_image(url, map_name, map_type, base_dir):
    response = requests.get(url)
    if response.status_code == 200:
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.join(base_dir), exist_ok=True)
        
        # Save the image
        with open(os.path.join(base_dir, f"{map_name}_{map_type}.png"), 'wb') as f:
            f.write(response.content)
        print(f"Downloaded image for {map_name} ({map_type})")
    else:
        print(f"Failed to download image for {map_name} ({map_type})")

# Base directory for saving images
base_dir = './data/fwi_fbp/images/current'        
        
# Base URL for the images
base_url = "https://cwfis.cfs.nrcan.gc.ca/geoserver/public/wms?service=WMS&version=1.1.0&request=GetMap"

# Define parameters for each map
map_parameters = [
    {"map_name": "Fire Danger", "map_type": "fdr", "bbox": "-2378164,-707617,3039835,3854382"},
    {"map_name": "Fire Weather Index", "map_type": "fwi", "bbox": "-2378164,-707617,3039835,3854382"},
    {"map_name": "Fire Type", "map_type": "ft_current", "bbox": "-2378164,-707617,3039835,3854382"}
]

# Loop through map parameters and download images
for params in map_parameters:
    map_name = params["map_name"]
    map_type = params["map_type"]
    bbox = params["bbox"]
    url = f"{base_url}&layers=public:{map_type}&styles=&bbox={bbox}&width=768&height=646&srs=EPSG:3978&format=image/png"
    download_image(url, map_name, map_type, base_dir)

Downloaded image for Fire Danger (fdr)
Downloaded image for Fire Weather Index (fwi)
Downloaded image for Fire Type (ft_current)


### 4. Using OWS service to collect features from the images gathered over the last 7 days
* Considering the following link to gather the most recent data for the features: https://cwfis.cfs.nrcan.gc.ca/geoserver/public/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=public:hotspots_last7days&maxFeatures=100000&outputFormat=CSV
* Download the hotspots_last7days.csv file  

* Please consider the documentation available at https://cwfis.cfs.nrcan.gc.ca/downloads/CWFIS_DataServices_HowtoAccessDailyMaps&DataLayers.pdf for OWS service

In [12]:
import requests

# Function to download and save CSV file from URL
def download_csv(base_dir, url):
    response = requests.get(url)
    if response.status_code == 200:
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.join(base_dir), exist_ok=True)
        
        with open(os.path.join(base_dir, "hotspots_last7days.csv"), 'wb') as f:
            f.write(response.content)
        print("Downloaded CSV file: hotspots_last7days.csv")
    else:
        print("Failed to download CSV file")
        
# Base directory for saving csv file
base_dir = './data/fwi_fbp/csv/'  

# Service URL for the CSV file
service_url = "https://cwfis.cfs.nrcan.gc.ca/geoserver/public/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=public:hotspots_last7days&maxFeatures=100000&outputFormat=CSV"

# Download the CSV file
download_csv(base_dir, service_url)

Downloaded CSV file: hotspots_last7days.csv


### 5. Gathering FWI/FBP historical data over 5-10 years

* Convert vector data to GeoJSON and CSV files
* Follow the same steps for other hotspot ZIP files for each year as described in section 1
* Download the GeoJSON and CSV files to the local path './data/fwi_fbp'

In [ ]:
import shapefile
import json
import csv
import os

# Function to convert shapefile shape and record to GeoJSON format
def shape_record_to_geojson(shape_record, fields):
    shape = shape_record.shape
    # Convert the record to a dictionary, mapping field names to values
    record = dict(zip(fields, shape_record.record))
    return {
        "type": "Feature",
        "geometry": shape.__geo_interface__,
        "properties": record
    }

# Function to read shapefile and convert to GeoJSON features
def read_shapefile_to_geojson_features(shp_path):
    reader = shapefile.Reader(shp_path)
    fields = [field[0] for field in reader.fields[1:]]
    return [shape_record_to_geojson(sr, fields) for sr in reader.shapeRecords()]

# Function to write GeoJSON data to file
def write_geojson(geojson, output_file_path):
    with open(output_file_path, 'w') as f:
        json.dump(geojson, f)

# Function to write CSV data to file
def write_csv(features, fields, output_file_path):
    with open(output_file_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fields)
        writer.writeheader()
        for feature in features:
            writer.writerow(feature['properties'])

# List of years to read shapefiles from
years = range(2014, 2024)

# Base directory path for the shapefiles
base_dir = './data/fwi_fbp'

# Loop through each year, read the corresponding shapefile, and save as GeoJSON and CSV
for year in years:
    shp_path = os.path.join(base_dir, f'{year}_hotspots', f'{year}_hotspots.shp')
    features = read_shapefile_to_geojson_features(shp_path)
    
    # Create GeoJSON FeatureCollection
    geojson = {
        "type": "FeatureCollection",
        "features": features
    }
    
    # Setting GeoJSON and output file paths
    json_dir = './data/fwi_fbp/GeoJSON'
    csv_dir = './data/fwi_fbp/csv'
    json_output_file_path = os.path.join(json_dir, f'{year}_hotspots.geojson')
    csv_output_file_path = os.path.join(csv_dir, f'{year}_hotspots.csv')
    
    # Write to GeoJSON file
    write_geojson(geojson, json_output_file_path)
    
    # Get field names for CSV
    fields = list(features[0]['properties'].keys())
    
    # Write to CSV file
    write_csv(features, fields, csv_output_file_path)
    
    print(f'Saved GeoJSON for year {year} to {json_output_file_path}')
    print(f'Saved CSV for year {year} to {csv_output_file_path}')